<a href="https://colab.research.google.com/github/YanivAmir/stock-predicition-CNN/blob/main/gather_historical_stock_data_from_tradingview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tvDatafeed import TvDatafeed, Interval
import yfinance as yf
import yahoo_fin.stock_info as si
from datetime import timedelta
import matplotlib.pyplot as plt
import pickle
import random
import datetime
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import MinMaxScaler
import logging
import re
tv = TvDatafeed("janivamir","########")

In [ ]:
savePath ='/path/to/save/data/'
other_tickers = si.tickers_other()
nasdaq_tickers = si.tickers_nasdaq()

In [ ]:
def gatherData(tickers,exchange,year, month, day,idx_start):
    date = str(day)+str(month)+str(year)
    data={}
    for idx,ticker in enumerate(tickers):
        if idx%100==0:
            print('\n'+str(idx),end = '\r', flush=True)
        if idx%500==0:
            print('\nsaving block '+str(idx_start+idx))
            with open(savePath+'TVstockData_'+exchange+'_'+date+'_'+str(idx_start+idx)+'.pickle', 'wb') as handle:
                pickle.dump(data, handle)
            data={}
        if len(ticker) <5: #no small caps with tickers longer than 4 chars
            print(ticker, end = '\r', flush=True)
            logging.getLogger('tvDatafeed.main').setLevel(logging.CRITICAL)
            data[ticker] = tv.get_hist(symbol=ticker,exchange=exchange,interval=Interval.in_1_hour,n_bars=5000) #5000 is the maximum time point allowed
            if data[ticker] is None or data[ticker].index[-1].date() != datetime.date(year, month, day): # remove tickers that are not recent
                del data[ticker]
                continue
            data[ticker]['cap'] = data[ticker].close * data[ticker].volume #market cap
            data[ticker]['wide_range'] = np.abs(data[ticker].high - data[ticker].low) / data[ticker].open  # movement volatility total as fraction of price
            data[ticker]['narrow_range/wide_range'] = np.abs(data[ticker].close - data[ticker].open) / np.abs(data[ticker].high - data[ticker].low)  # movement final volatility as fraction of total volatility
            data[ticker].fillna(0, inplace=True)

    print('\nfinal '+str(idx_start+idx+1))
    with open(savePath+'TVstockData_'+exchange+'_'+date+'_final_'+str(idx_start+idx)+'.pickle', 'wb') as handle:
        pickle.dump(data, handle)
    return data


In [ ]:
idx_start=0
data = gatherData(other_tickers,'NYSE',2024,9,17,0)

idx_start=3000
tickers_left= other_tickers[idx_start:]
data = gatherData(tickers_left,'NYSE',2024,9,17,idx_start)

idx_start = 0
data = gatherData(nasdaq_tickers,'NASDAQ',2024,9,18,idx_start)

idx_start = 3700
data = gatherData(nasdaq_tickers[idx_start:],'NASDAQ',2024,9,20,idx_start)